In [ ]:
%pip install pymongo tqdm sentence-transformers

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
import pandas as pd
import json
from collections import Counter
from tqdm import tqdm
from pymongo import MongoClient
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
conf = pyspark.SparkConf()


import copy
import numpy as np

from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


tqdm.pandas()

MONGO_CONN = 'mongodb+srv://<username>:<password>@retail-demo.2wqno.mongodb.net/?retryWrites=true&w=majority'

In [ ]:
@F.udf()
def get_vec(text):
    return model.encode(text)

In [ ]:
catalog_status = spark.readStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_myn").\
            option('spark.mongodb.change.stream.publish.full.document.only','true').\
            option('spark.mongodb.aggregation.pipeline',[]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

# update atp status to final collection based on other required fields
# catalog_status = catalog_status.filter(((F.col("articleType").isNotNull()) & (~F.col("articleType").cast("String").like("null"))))\
#         .filter(((F.col("mfg_brand_name").isNotNull()) & (~F.col("mfg_brand_name").cast("String").like("null")))) \
#         .filter(((F.col("title").isNotNull()) & (~F.col("title").cast("String").like("null")))) \
#         .filter(((F.col("score").isNotNull()) & (~F.col("score").cast("String").like("null")))) \
#         .filter(((F.col("price").isNotNull()) & (~F.col("price").cast("String").like("null")))) \

catalog_status = catalog_status.withColumn("discountedPrice", F.col("price") * F.col("pred_price"))

catalog_status = catalog_status.withColumn("atp", (F.col("atp").cast("boolean") & F.lit(1).cast("boolean")).cast("integer"))

catalog_status.withColumn("vec", get_vec("title"))

catalog_status = catalog_status.drop("_id")
catalog_status.writeStream.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_final_myn").\
            option('spark.mongodb.operationType', "update").\
            option('spark.mongodb.idFieldList', "id").\
            option("forceDeleteTempCheckpointLocation", "true").\
            option("checkpointLocation", "/tmp/retail-atp-myn5/_checkpoint/").\
            outputMode("append").\
            start()

